<a href="https://colab.research.google.com/github/jankovicsandras/ml/blob/main/HF_llama2_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Init

In [23]:
! ls -la
! pwd

total 5379436
drwxr-xr-x 1 root root       4096 Aug 24 06:34 .
drwxr-xr-x 1 root root       4096 Aug 24 06:10 ..
drwxr-xr-x 4 root root       4096 Aug 22 13:38 .config
-rw-r--r-- 1 root root 5508521088 Jul 18 18:05 llama-2-13b-chat.ggmlv3.q2_K.bin
drwxr-xr-x 1 root root       4096 Aug 22 13:39 sample_data
/content


Model download

In [14]:
! wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q2_K.bin

--2023-08-24 06:34:25--  https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q2_K.bin
Resolving huggingface.co (huggingface.co)... 13.35.166.50, 13.35.166.69, 13.35.166.36, ...
Connecting to huggingface.co (huggingface.co)|13.35.166.50|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/cd/43/cd4356b11767f5136b31b27dbb8863d6dd69a4010e034ef75be9c2c12fcd10f7/de25498144f05fd3ee41cd2250c16f23a8415a4a4c9f4c1df1a3efd9b3c0991d?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-13b-chat.ggmlv3.q2_K.bin%3B+filename%3D%22llama-2-13b-chat.ggmlv3.q2_K.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1693118065&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MzExODA2NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9jZC80My9jZDQzNTZiMTE3NjdmNTEzNmIzMWIyN2RiYjg4NjNkNmRkNjlhNDAxMGUwMzRlZjc1YmU5YzJ

Install deps


In [6]:
!pip -q install langchain transformers huggingface_hub accelerate bitsandbytes llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.3 MB/s eta 0:00:00


Import, API token init

In [1]:
import os
#from langchain import PromptTemplate, LLMChain, HuggingFaceHub
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()

··········


Env

In [3]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

PromptTemplate

In [5]:
from langchain import PromptTemplate #, LLMChain, HuggingFaceHub

template = """Question: {question}
Answer: Let's think step by step. """

prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

Model

In [15]:
from langchain import HuggingFaceHub, LLMChain, LlamaCpp
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer

model_name = "llama-2-13b-chat.ggmlv3.q2_K.bin"
llm = LlamaCpp(
	model_path=model_name #, model_kwargs={"temperature":0.1, "max_length":128}
)

#m = AutoModelForCausalLM.from_pretrained(model_name)


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


LangChain LLMChain run

In [22]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
#question = "What is the number of the days of the week squared?"  # correct first try from llama-2-13b-chat.ggmlv3.q2_K.bin
#question = "How many fingers are there on six hands?" # correct first try from llama-2-13b-chat.ggmlv3.q2_K.bin
question = """Andy has four apples. Bill has seven oranges. Claire has three apples.
Daniel has two oranges. Sort the people by the number of their fruits after Andy gives Claire two and Bill gives Daniel four.""" # wrong first try from llama-2-13b-chat.ggmlv3.q2_K.bin
# A=4 B=7 C=3 D=2 ; A-2 C+2 B-4 D+4 ; A=2 B=3 C=5 D=6 ; ABCD or DCBA is correct

result = llm_chain(question)

print(result['text'])
print(result)

Llama.generate: prefix-match hit



First, Andy gives Claire two apples, so she now has two + three = five apples.
Next, Bill gives Daniel four oranges, so Daniel now has two + four = six oranges. 
Now, let's compare the total number of fruits for each person: 
Claire - five apples 
Bill - six oranges 
Daniel - six oranges and two apples (Andy gave him two apples)  
So, the order from highest to lowest is:
Claire, Bill, Daniel.
{'question': 'Andy has four apples. Bill has seven oranges. Claire has three apples. \nDaniel has two oranges. Sort the people by the number of their fruits after Andy gives Claire two and Bill gives Daniel four.', 'text': "\nFirst, Andy gives Claire two apples, so she now has two + three = five apples.\nNext, Bill gives Daniel four oranges, so Daniel now has two + four = six oranges. \nNow, let's compare the total number of fruits for each person: \nClaire - five apples \nBill - six oranges \nDaniel - six oranges and two apples (Andy gave him two apples)  \nSo, the order from highest to lowest i